In [ ]:
import pandas as pd
from pathlib import Path
import pyarrow.parquet as pq
from dataclasses import dataclass
import hvplot.pandas 
# hv.renderer('bokeh').theme = 'dark_minimal'

# Data Loading
It is always a good advice to use standard formats to transfer data from one computer to another. One of the most used formats is parquet. The following code should help you to load the data into your python environment.

In [ ]:
dataset_root = Path(r"../data") # Raw string works without escaping \


@dataclass
class Case():
    info: pd.DataFrame
    measurements: pd.DataFrame


class RawDataset():

    def __init__(self, root, unit = "VG4", load_training=False, load_synthetic=False) -> None:
        
        
        read_pq_file = lambda f: pq.read_table(root / f).to_pandas()
        
        
        cases = {
            "test": [f"{unit}_generator_data_testing_real_measurements.parquet", root / f"{unit}_generator_data_testing_real_info.csv" ], 
        }
        
        if load_training:
            cases = {
                **cases,
                "train": [f"{unit}_generator_data_training_measurements.parquet", root / f"{unit}_generator_data_training_info.csv" ], 
            }
        
        if load_synthetic:
            cases = {
                **cases,
                "test_s01": [f"{unit}_generator_data_testing_synthetic_01_measurements.parquet", root / f"{unit}_generator_data_testing_synthetic_01_info.csv"], 
                "test_s02": [f"{unit}_generator_data_testing_synthetic_02_measurements.parquet", root / f"{unit}_generator_data_testing_synthetic_02_info.csv"]
            }
        
        
        self.data_dict = dict()
        
        for id_c, c in cases.items():
            # if you need to verify the parquet header:
            # pq_rows = RawDataset.read_parquet_schema_df(root / c[0])
            info = pd.read_csv(c[1])
            measurements = read_pq_file(c[0])
            self.data_dict[id_c] = Case(info, measurements)
            
        
        
    @staticmethod
    def read_parquet_schema_df(uri: str) -> pd.DataFrame:
        """Return a Pandas dataframe corresponding to the schema of a local URI of a parquet file.

        The returned dataframe has the columns: column, pa_dtype
        """
        # Ref: https://stackoverflow.com/a/64288036/
        schema = pq.read_schema(uri, memory_map=True)
        schema = pd.DataFrame(({"column": name, "pa_dtype": str(pa_dtype)} for name, pa_dtype in zip(schema.names, schema.types)))
        schema = schema.reindex(columns=["column", "pa_dtype"], fill_value=pd.NA)  # Ensures columns in case the parquet file has an empty dataframe.
        return schema
    



rds_u4 = RawDataset(dataset_root, "VG4", load_synthetic=False, load_training=True)
rds_u5 = RawDataset(dataset_root, "VG5", load_synthetic=True, load_training=True)
rds_u6 = RawDataset(dataset_root, "VG6", load_synthetic=True)




# Operation Modes

As you can see, the operation modes (at the very bottom) are not in the info schema, but you can find them when you explore the parquet schema (metadata of the parquet data file).

In [ ]:
RawDataset.read_parquet_schema_df(dataset_root / "VG4_generator_data_testing_real_measurements.parquet")

We can use `head()` to glance at the data.

In [ ]:
rds_u4.data_dict["train"].info.head()

In [ ]:
rds_u4.data_dict["train"].measurements.head()

# Data Exploration
- Make sure that you explore the data in detail. For example, we will see that there is quite a big gap in operation from train to test if we look at the injector pressures of unit 4.
- To not overload our machine when plotting, we can use an index selection trick to downsample the data (since the dataframes are quite large) `[::100]`.
    - Please be aware that this is just to get some insights, we are only plotting every 100th step.

In [ ]:
rds_u5.data_dict["train"].info

There are quite many signals, so it makes sense to study them in groups. Here we focus on the temperatures `_tmp`), but you are supposed to look at all the other signals as well of course!

In [ ]:
df = rds_u4.data_dict["train"].info
temperature_attrs = df[df.attribute_name.str.endswith("_tmp")]
temperature_attrs

We can use pandas to directly plot the data. Usually, panda uses matplotlib, but when we install the hvplot module, we can have a much nicer html data explorer.

In [ ]:
rds_u4.data_dict["train"].measurements[::100].plot()

In [ ]:
rds_u4.data_dict["train"].measurements[::100].hvplot.explorer(y=[v for v in temperature_attrs.attribute_name.values])

An other example would be to look at injector pressures:

In [ ]:
df = rds_u4.data_dict["train"].info
temperature_attrs = df[df.attribute_name.str.contains("injector")]
temperature_attrs

In [ ]:
rds_u4.data_dict["train"].measurements[::100].hvplot.explorer(y=["injector12_pressure", "injector34_pressure"])

As you can see, the test set is quite short compared to the training set. For injector 1, the test distribution is different compared to the training distribution (distribution shift).
Be aware that such distribution shifts can greatly influence the performance of your model.

In [ ]:
rds_u4.data_dict["test"].measurements[::100].hvplot.explorer(y=["injector12_pressure", "injector34_pressure"])

We can also explore the machines in operation next to each other:

The training data sets.

In [ ]:
plt = (
    rds_u4.data_dict["train"].measurements[::1000].hvplot()
    + rds_u5.data_dict["train"].measurements[::1000].hvplot()
    ).cols(1)

plt

We see that the test datasets are recorded troughout different seasons, which can have an impact on the recorded data.

In [ ]:
plt = (
    rds_u4.data_dict["test"].measurements[::1000].hvplot()
    + rds_u5.data_dict["test"].measurements[::1000].hvplot()
    + rds_u6.data_dict["test"].measurements[::1000].hvplot()
    ).cols(1)

plt

Now it's your turn. Try to explore the data as much as possible before engineering the solution. The main difficulty in real-world problems often isn't the complexity, but understanding the actual problem you are trying to solve, as there is no "guidance" like in the exercises you saw throughout this class. Remember, there are always trade-offs when selecting a model which you should evaluate before investing a lot of time on the implementation.

# Data Exploration of VG4

In [ ]:
rds_u4 = RawDataset(dataset_root, "VG4", load_synthetic=False, load_training=True)

In [ ]:
vg4_train, vg4_test = rds_u4.data_dict['train'], rds_u4.data_dict['test']

In [ ]:
# Checking the structure of the measurements data
print("VG4 Training Data Structure:\n")
print(vg4_train.measurements.info())  # Overview of columns and types

# Checking metadata for additional information on each variable
print("\nVG4 Training Metadata Structure:\n")
print(vg4_train.info.head())  # Displaying metadata to see attributes for each signal

## Missing Data

In [ ]:
# Checking for missing values in the measurements data
missing_data = vg4_train.measurements.isnull().sum()
print("\nMissing Data Summary:\n")
print(missing_data[missing_data > 0])  # Only displaying columns with missing values

## Operating Modes

In [ ]:
# Exploring operating modes and their distributions
operating_modes = ['turbine_mode', 'pump_mode', 'machine_on', 'machine_off', 'equilibrium_turbine_mode', 'equilibrium_pump_mode', 'dyn_only_on']
mode_counts = vg4_train.measurements[operating_modes].sum()
print("\nOperating Modes Distribution:\n")
print(mode_counts)

## Grouping Variables

In [ ]:
# First, let's identify all columns in the measurements data
all_columns = vg4_train.measurements.columns

# Separate features into categories
non_boolean_non_operating = []  # Stores numeric features that aren't booleans or operating conditions

for col in all_columns:
    if col not in operating_modes:
        # Check if column values are boolean-like (True/False or binary 0/1)
        unique_vals = vg4_train.measurements[col].dropna().unique()
        if len(unique_vals) > 2 or not set(unique_vals).issubset({0, 1, True, False}):
            non_boolean_non_operating.append(col)
        else:
            print('Non-unique or boolean feature:', col)

print("Non-boolean, Non-operating Condition Features:")
print(non_boolean_non_operating)

all_features = vg4_train.measurements[non_boolean_non_operating]

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Calculate the correlation matrix for the measurements data
correlation_matrix = all_features.corr()

# Plotting correlation matrix as a heatmap
plt.figure(figsize=(12, 10))
sns.heatmap(correlation_matrix, annot=False, cmap='coolwarm', cbar=True)
plt.title("Correlation Matrix for VG4 Measurements")
plt.show()

In [ ]:
from scipy.cluster.hierarchy import linkage, dendrogram, fcluster
from scipy.spatial.distance import squareform
import numpy as np
import matplotlib.pyplot as plt

correlation_matrix = np.abs(correlation_matrix)

# Convert correlation matrix to distance matrix for clustering (1 - correlation)
distance_matrix = 1 - correlation_matrix

# Perform hierarchical clustering using the distance matrix
linked = linkage(squareform(distance_matrix), method='ward')

# Create a dendrogram
plt.figure(figsize=(10, 7))
dendrogram(linked, labels=correlation_matrix.index)
plt.title('Dendrogram of Asset Clusters')
plt.xlabel('Assets (ID)')
plt.ylabel('Distance')
plt.show()

# Clustered heatmap of the correlation matrix
plt.figure(figsize=(40, 30))
sns.clustermap(correlation_matrix, method='ward', cmap='coolwarm', annot=False)
plt.title('Clustered Heatmap of Asset Correlations')
plt.show()

## Visualize some features

In [ ]:
from sklearn.preprocessing import StandardScaler

# Initialize the scaler
scaler = StandardScaler()

# Select only the numerical columns for scaling
# We'll ignore boolean columns and any columns that aren't numerical
numerical_columns = non_boolean_non_operating

# Fit the scaler on the training set and transform it
vg4_train_standardized = vg4_train.measurements.copy()  # Creating a copy to avoid modifying the original data
vg4_train_standardized[numerical_columns] = scaler.fit_transform(vg4_train.measurements[numerical_columns])

# Transform the test set using the same scaler (we do not refit to ensure consistency)
vg4_test_standardized = vg4_test.measurements.copy()  # Copying test data
vg4_test_standardized[numerical_columns] = scaler.transform(vg4_test.measurements[numerical_columns])

In [ ]:
# Define important features to compare between train and test sets
important_features = non_boolean_non_operating #[col for col in vg4_train.measurements.columns if 'tmp' in col]

# Prepare a dictionary to store the plots for each feature
comparison_plots = {}

# Generate distribution plots for each feature in the list
for feature in important_features:
    # Check if feature exists in both train and test datasets
    if feature in vg4_train_standardized.columns and feature in vg4_test_standardized.columns:
        train_data = vg4_train_standardized[feature].dropna()
        test_data = vg4_test_standardized[feature].dropna()

        # Overlay normalized histograms for train and test distributions
        comparison_plot = train_data.hvplot.hist(
            bins=30, alpha=0.5, width=500, height=400, color='blue', 
            ylabel='Density', xlabel=f'{feature} Value', label='Train', 
            normed=True  # Normalize the histogram
        ) * test_data.hvplot.hist(
            bins=30, alpha=0.5, color='orange', label='Test', 
            normed=True  # Normalize the histogram
        )

        # Add title and legend
        comparison_plot = comparison_plot.opts(
            title=f'Normalized pdf of {feature}',
            legend_position='top_right',
            ylim=(0, 1)  # Set y-axis limit to 1
        )
        
        # Store each plot in a dictionary for easy access and display
        comparison_plots[feature] = comparison_plot

# Display the plots as a grid layout
hv.Layout(comparison_plots.values()).cols(2)